## Preparação:
- remoção do postgres caso ele já esteja no computador.

In [ ]:
#apagando postgres caso ele já esteja no compouter
!sudo systemctl stop postgresql 
!sudo apt-get --purge remove postgresql*
!sudo apt-get autoremove
!sudo apt-get autoclean
!psql --version


- clonagem do repositório, junto dos comandos iniciais

In [ ]:
!git clone https://github.com/foliveirafilho/tpch-pgsql.git
!wget -q https://github.com/electrum/tpch-dbgen/archive/32f1c1b92d1664dba542e927d23d86ffa57aa253.zip -O tpch-dbgen.zip
!unzip -q tpch-dbgen.zip && mv tpch-dbgen-32f1c1b92d1664dba542e927d23d86ffa57aa253 tpch-dbgen && rm tpch-dbgen.zip


- reinstalação do postgres

In [ ]:
!sudo apt-get install -y postgresql postgresql-contrib


- verificação das versões

In [ ]:
!gcc --version
!python3 --version
!pg_config --version


## Passo a passo repositório

- instalação do programa 

In [ ]:
!pip3 install -r tpch-pgsql/requirements.txt

- Criação de usuário no postgres

In [ ]:
!sudo -u postgres createuser tpch
!sudo -u postgres createdb tpchdb
!sudo -u postgres psql -c "ALTER USER tpch WITH ENCRYPTED PASSWORD '1234';"
!sudo -u postgres psql -c "GRANT ALL PRIVILEGES ON DATABASE tpchdb TO tpch;"
!PGPASSWORD=1234 psql -h localhost -p 5432 -d tpchdb -U tpch -c "\l"


could not change directory to "/home/zico/Documentos/ufam/bd/bd/tp3": Permissão negada


- rodando o programa (comparar com a versão do jhon)

In [ ]:
!python3 tpch_pgsql.py prepare -H localhost -p 5432 -U tpch -W 1234 -d tpchdb -i ./data
#executando  com tpch-dbgen dentro de tpch-pgsql
#deu certo
#passo 3

!python3 tpch_pgsql.py load -H localhost -p 5432 -U tpch -W 1234 -d tpchdb
#passo 3

!python3 tpch_pgsql.py query -H localhost -p 5432 -U tpch -W 1234 -d tpchdb
#passo 3

## Tarefa 4: identificação do sistema

In [72]:
!echo "--------------------NOME PROCESS--------------"
!lscpu | grep "Nome"
!echo ""
!echo "---------------RAM------------------"
!free -h # Detalhes sobre a memória RAM tarefa 4
!echo ""
!echo "---------------Disco------------------"
!lsblk -o NAME,SIZE,TYPE | grep "disk" # Detalhes sobre o disco tarefa 4
!echo ""
!echo "---------------cache------------------"
!lscpu | grep -i "cache"  # Cache do processador tarefa 4
!echo ""
!echo "---------------Distro------------------"
!lsb_release -a | grep "Distributor" # Detalhes da distribuição e versão do Linux tarefa 4
!echo ""
!echo "---------------SO e Kernel------------------"
!uname -a # Informações sobre o Sistema Operacional e Kernel tarefa 4

--------------------NOME PROCESS--------------


Nome do modelo:                       12th Gen Intel(R) Core(TM) i5-12450H

---------------RAM------------------
               total       usada       livre    compart.  buff/cache  disponível
Mem.:          7,5Gi       2,9Gi       728Mi       859Mi       3,8Gi       3,3Gi
Swap:          2,0Gi       1,1Gi       928Mi

---------------Disco------------------
nvme0n1     238,5G disk

---------------cache------------------
cache de L1d:                         320 KiB (8 instances)
cache de L1i:                         384 KiB (8 instances)
cache de L2:                          7 MiB (5 instances)
cache de L3:                          12 MiB (1 instance)

---------------Distro------------------
No LSB modules are available.
Distributor ID:	Zorin

---------------SO e Kernel------------------
Linux Mengo 6.8.0-48-generic #48~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Mon Oct  7 11:24:13 UTC 2 x86_64 x86_64 x86_64 GNU/Linux


## Tarefa 5:Verificação de Parâmetros de Armazenamento (Adaptada para SSD)


In [73]:
!echo "---------------Capacidade Disco------------------"
!lsblk -o NAME,SIZE,TYPE | grep "disk" 

---------------Capacidade Disco------------------
nvme0n1     238,5G disk


In [ ]:
!sudo apt-get install smartmontools 
#programa para rodar o próximo comando


In [52]:
# Verificando informações de latência do SSD usando smartctl
!sudo smartctl -a /dev/nvme0n1 #mudar o /dev/... pelo nome do ssd do computador

smartctl 7.2 2020-12-30 r5155 [x86_64-linux-6.8.0-48-generic] (local build)
Copyright (C) 2002-20, Bruce Allen, Christian Franke, www.smartmontools.org

=== START OF INFORMATION SECTION ===
Model Number:                       SM2P41C8-256GC1
Serial Number:                      KO022L1KKNGD
Firmware Version:                   V1223C0
PCI Vendor/Subsystem ID:            0x1cc1
IEEE OUI Identifier:                0x707c18
Total NVM Capacity:                 256.060.514.304 [256 GB]
Unallocated NVM Capacity:           0
Controller ID:                      1
NVMe Version:                       1.4
Number of Namespaces:               1
Namespace 1 Size/Capacity:          256.060.514.304 [256 GB]
Namespace 1 Formatted LBA Size:     512
Namespace 1 IEEE EUI-64:            707c18 0002020100
Local Time is:                      Mon Nov 11 15:52:42 2024 -04
Firmware Updates (0x16):            3 Slots, no Reset required
Optional Admin Commands (0x0017):   Security Format Frmw_DL Self_Test
Optional 

In [32]:
# Taxa de transferência de leitura do SSD
!sudo hdparm -tT /dev/nvme0n1



/dev/nvme0n1:
 Timing cached reads:   10272 MB in  2.00 seconds = 5142.22 MB/sec
 Timing buffered disk reads: 3318 MB in  3.00 seconds = 1105.80 MB/sec


In [33]:
# Verificando a vida útil (TBW) do SSD
!sudo smartctl -A /dev/nvme0n1 | grep "Data Units Written"


Data Units Written:                 4.781.530 [2,44 TB]


In [ ]:
# Tipo de memória utilizada no SSD (se disponível)
!sudo smartctl -i /dev/nvme0n1
#ultima 5. a)

smartctl 7.2 2020-12-30 r5155 [x86_64-linux-6.8.0-48-generic] (local build)
Copyright (C) 2002-20, Bruce Allen, Christian Franke, www.smartmontools.org

=== START OF INFORMATION SECTION ===
Model Number:                       SM2P41C8-256GC1
Serial Number:                      KO022L1KKNGD
Firmware Version:                   V1223C0
PCI Vendor/Subsystem ID:            0x1cc1
IEEE OUI Identifier:                0x707c18
Total NVM Capacity:                 256.060.514.304 [256 GB]
Unallocated NVM Capacity:           0
Controller ID:                      1
NVMe Version:                       1.4
Number of Namespaces:               1
Namespace 1 Size/Capacity:          256.060.514.304 [256 GB]
Namespace 1 Formatted LBA Size:     512
Namespace 1 IEEE EUI-64:            707c18 0002020100
Local Time is:                      Mon Nov 11 15:22:11 2024 -04



In [43]:
# Verificando o modo de leitura e gravação do SSD usando hdparm
!sudo hdparm /dev/nvme0n1 



/dev/nvme0n1:
 readonly      =  0 (off)
 readahead     = 256 (on)
 geometry      = 244198/64/32, sectors = 500118192, start = 0


In [45]:
# Verificando a ativação do TRIM no SSD usando smartctl
!sudo smartctl -i /dev/nvme0n1 | grep "TRIM"


In [48]:

# Verificar TRIM usando hdparm
!sudo hdparm -I /dev/nvme0n1 | grep "TRIM"